**Importar biliotecas y leer datos**

In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob

In [3]:
# Importamos los archivos y los almacenamos como DataFrames

df_steam_games = pd.read_csv('data\steam_games.csv')
df_reviews = pd.read_csv(r'data\reviews.csv')
df_items = pd.read_csv('data\items.csv')

**Extracción, transformación y carga de datos (ETL)**

**df_reviews**

In [4]:
# Visualizamos el dataframe
df_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


In [5]:
# Visualizamos las dimensiones del df
df_reviews.shape

(59305, 5)

In [6]:
# Visualizamos las columnas existentes
df_reviews.columns

Index(['user_id', 'item_id', 'posted', 'recommend', 'review'], dtype='object')

In [7]:
# Visualizamos el tipo de datos de las columnas
df_reviews.dtypes

user_id      object
item_id       int64
posted       object
recommend      bool
review       object
dtype: object

In [8]:
# Revisamos datos nulos por columna
datos_no_nulos = df_reviews.count()
datos_nulos = df_reviews.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
user_id,59305,0
item_id,59305,0
posted,59305,0
recommend,59305,0
review,59275,30


In [9]:
# Definición de la función

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity < 0:
        return 0  # Sentimiento negativo
    elif analysis.sentiment.polarity == 0:
        return 1  # Sentimiento neutral
    else:
        return 2  # Sentimiento positivo

In [10]:
# Transformando tipo de dato float a str para la columna 'review' debido a que el análisis de sentimiento arrojaba un error por el tipo de dato
df_reviews['review'] = df_reviews['review'].astype(str)

In [11]:
# Aplicar la función de análisis de sentimiento a la columna 'texto' y crear 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analizar_sentimiento)

In [12]:
# Transformando tipo de dato float a str para la columna 'sentiment_analysis' debido a que el análisis de sentimiento arrojaba un error por el tipo de dato
df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].astype(str)

In [13]:
# Eliminamos la columna 'review'
df_reviews = df_reviews.drop('review', axis=1)

In [14]:
df_reviews.head()

,user_id,item_id,posted,recommend,sentiment_analysis
0,76561197970982479,1250,"Posted November 5, 2011.",True,2
1,76561197970982479,22200,"Posted July 15, 2011.",True,2
2,76561197970982479,43110,"Posted April 21, 2011.",True,2
3,js41637,251610,"Posted June 24, 2014.",True,2
4,js41637,227300,"Posted September 8, 2013.",True,0


In [15]:
# Transformamos el tipo de dato de posted para que sea en un formato de fecha

# Extraer la parte de la fecha de la columna "posted" usando una expresión regular
df_reviews['posted'] = df_reviews['posted'].str.extract(r'Posted (.+?)\.')

# Definir un formato que permita fechas con o sin año
date_formats = ['%B %d, %Y', '%B %d']

# Intentar convertir la columna "posted" al formato deseado
for date_format in date_formats:
    try:
        df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format=date_format, errors='coerce')
        break  # Salir del bucle si la conversión es exitosa
    except ValueError:
        pass  # Continuar con el siguiente formato si hay un error

# Reemplazar el formato de fecha con el deseado
df_reviews['posted'] = df_reviews['posted'].dt.strftime('%Y-%m-%d')

**df_steam_games**

In [16]:
# Visualizamos el dataframe
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域
4,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,"['Single-player', 'Steam Achievements']",3.99,False,772540.0,Trickjump Games Ltd


In [17]:
# Visualizamos las dimensiones del df
df_steam_games.shape

(28852, 13)

In [18]:
# Visualizamos las columnas existentes
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [19]:
# Visualizamos el tipo de datos de las columnas
df_steam_games.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access       bool
id              float64
developer        object
dtype: object

In [20]:
# Consideramos que columnas eliminar de df_steam_games

# app_name: Contiene los mismos datos que 'title'
# tags: contiene caracteristicas similares que 'genres'
# url: No contiene información que pueda aportar al modelo
# reviews_url: No posee información importante

In [21]:
columnas_eliminar = ['app_name', 'tags', 'url', 'reviews_url']

df_steam_games = df_steam_games.drop(columns=columnas_eliminar)

In [22]:
# Chequeamos las columnas que quedaron
df_steam_games.columns

Index(['publisher', 'genres', 'title', 'release_date', 'specs', 'price',
       'early_access', 'id', 'developer'],
      dtype='object')

In [23]:
# Revisamos datos nulos por columna

datos_no_nulos = df_steam_games.count()
datos_nulos = df_steam_games.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
publisher,23936,4916
genres,28852,0
title,28851,1
release_date,28834,18
specs,28566,286
price,27622,1230
early_access,28852,0
id,28851,1
developer,28682,170


In [24]:
# Elimino el unico registro con id nulo
# Eliminar registros con valores nulos en 'columna_especifica'
df_steam_games = df_steam_games.dropna(subset=['id'])

In [25]:
# Transformo 'id' en un tipo de dato int 
df_steam_games['id'] = df_steam_games['id'].astype(int)

In [26]:
# Visualizamos el tipo de datos de las columnas
df_steam_games.dtypes

publisher       object
genres          object
title           object
release_date    object
specs           object
price           object
early_access      bool
id               int32
developer       object
dtype: object

In [27]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28851 entries, 0 to 28851
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     23935 non-null  object
 1   genres        28851 non-null  object
 2   title         28850 non-null  object
 3   release_date  28833 non-null  object
 4   specs         28565 non-null  object
 5   price         27621 non-null  object
 6   early_access  28851 non-null  bool  
 7   id            28851 non-null  int32 
 8   developer     28681 non-null  object
dtypes: bool(1), int32(1), object(7)
memory usage: 1.9+ MB


**df_items**

In [28]:
# Visualizamos el dataframe
df_items.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [29]:
# Visualizamos las dimensiones del df
df_items.shape

(5153209, 5)

In [30]:
# Visualizamos las columnas existentes
df_items.columns

Index(['user_id', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [31]:
# Visualizamos el tipo de datos de las columnas
df_items.dtypes

user_id             object
item_id              int64
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

In [32]:
# Revisamos datos nulos por columna

datos_no_nulos = df_items.count()
datos_nulos = df_items.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
user_id,5153209,0
item_id,5153209,0
item_name,5153209,0
playtime_forever,5153209,0
playtime_2weeks,5153209,0


**Exportación**

In [33]:
#optimizar datos

df_items = df_items[:300000]
df_reviews = df_reviews[:20000]
df_steam_games = df_steam_games[:20000]

In [34]:
# Con los dataframes transformados los exportamos en limpio para su posterior utilizacion

df_steam_games.to_csv('data\output\steam_games.csv', index=False)

In [35]:
df_reviews.to_csv(r'data\output\reviews.csv', index=False)

In [36]:
df_items.to_csv('data/output/items.csv', index=False)